In [41]:
import json
import os
import numpy as np
from astropy.io import fits

def get_trans(pwv, wres=250000):
	inputs = {
		"incl_moon": "N",
		"incl_loweratm": "N",
		"incl_upperatm": "N",
		"incl_airglow": "N",
		"incl_starlight": "N",
		"pwv": pwv,
		"wmin": 2200,
		"wmax": 2400,
		"wgrid_mode": "fixed_spectral_resolution",
		"wres": wres
	}

	with open("skycalc-inputs.json", "w") as fp:
		json.dump(inputs, fp)

	os.system("skycalc_cli -i skycalc-inputs.json -o outputs3.fits")
	with fits.open("outputs3.fits", memmap=False) as hdul:
		rect = hdul[1].data
	hdul.close()
	row = rect["trans"]
	return row

pwv_values = [0.5, 1.0, 1.5, 2.5, 3.5, 5.0, 7.5, 10.0, 20.0]

trans = []
for pwv in pwv_values:
	row = get_trans(pwv)
	trans.append(row)

trans = np.asarray(trans)
trans.shape

In [36]:
telGrid = jnp.load("../data/trGrid_crires.npy")
telGrid.shape

array([[0.91450541, 0.93732338, 0.94901085, ..., 0.97369355, 0.97856674,
        0.97472175],
       [0.90604027, 0.93088946, 0.9441435 , ..., 0.97195438, 0.9766358 ,
        0.97280923],
       [0.89802767, 0.92467336, 0.93949795, ..., 0.97006336, 0.97467153,
        0.97078064],
       ...,
       [0.80915115, 0.853905  , 0.88575979, ..., 0.941359  , 0.94501341,
        0.94020598],
       [0.7742725 , 0.82595587, 0.86430552, ..., 0.92750706, 0.93049543,
        0.92505101],
       [0.65653646, 0.72683868, 0.78497213, ..., 0.86000344, 0.86171804,
        0.85532773]])

In [39]:
(trans == telGrid).all()

True

In [42]:
trans = []
for pwv in pwv_values:
	row = get_trans(pwv, wres=1000000)
	trans.append(row)
	
trans = np.asarray(trans)

In [45]:
np.save("../data/trGrid_highRes.npy", trans)